# fpgaConvNet: CNN Model to Vivado IP Example

This tutorial demonstrates the end-to-end flow from a ONNX model to generated hardware. The LeNet network is used as an example. Let's start by looking at the base-line architecture through the lens of the [fpgaconvnet-model](github.com/AlexMontgomerie/fpgaconvnet-model) modelling part of the project.

In [1]:
from IPython.display import Image 
from fpgaconvnet.models.network import Network

# load network
net = Network("lenet", "models/lenet-sim.onnx")
#net.visualise("baseline.png", mode="png")

# display to jupyter notebook
#im = Image('baseline.png')
#display(im)

/home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load the zedboard platform details
net.update_platform("platforms/zedboard.json")

# show latency, throughput and resource predictions
print(f"predicted latency (us): {net.get_latency()*1000000}")
print(f"predicted throughput (img/s): {net.get_throughput()} (batch size={net.batch_size})")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")

predicted latency (us): 67490.07999999999
predicted throughput (img/s): 14.816992363914817 (batch size=1)
predicted resource usage: {'FF': 29338, 'LUT': 23113, 'DSP': 4, 'BRAM': 3387}


To increase performance, the [samo](github.com/AlexMontgomerie/samo) design-space exploration tool can be used. This tool generalises the CNN to FPGA mapping problem for Streaming Architectures. A command-line interface can be used to optimise a design. We will use the rule-based optimiser, and use a latency performance objective.

In [3]:
!mkdir -p outputs
import samo.cli

# invoking the CLI from python
samo.cli.main([
    "--model", "models/lenet-sim.onnx",
    "--platform", "platforms/zedboard.json",
    "--output-path", "outputs/lenet_opt.json",
    "--backend", "fpgaconvnet",
    "--optimiser", "rule",
    "--objective", "latency"
])

#### ( run settings ) ####
 * model    : models/lenet-sim.onnx
 * backend  : fpgaconvnet
 * platform : platforms/zedboard.json
 * optimiser : rule
 * objective : latency
 * batch size : 256
 * enable_reconf : true
 * seed : 2341833006
 * output_path : outputs/lenet_opt.json
##########################
Partition 0:
------------

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  ---------------
        400         12800  13200.0/34112.0


Layer                       Latency (cycles)    DSP    BRAM    LUT     FF
------------------------  ------------------  -----  ------  -----  -----
import/conv1first/Conv2D                3136    200     232  20286  25185


  Latency (cycles)  DSP        BRAM       LUT            FF
------------------  ---------  ---------  -------------  --------------
              3136  200 / 220  232 / 280  20286 / 53200  25185 / 106400



Partition 1:
------------

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  ---------------
  

  in (Mbps)    out (Mbps)  total (Mbps)
-----------  ------------  --------------------------
        100      0.159439  100.15943877551021/34112.0


Layer                       Latency (cycles)    DSP    BRAM    LUT     FF
------------------------  ------------------  -----  ------  -----  -----
import/conv2/Conv2D                   100352    100     198  26467  31927
import/conv2/Relu                      12544      0       0      0      0
import/pool2/MaxPool                   12544      0       3    903   1307
import/conv3/Conv2D                    65536     49    3286   8132  10134
import/conv3/Relu                       1024      0       0      0      0
import/conv4last/BiasAdd               10240      1      13   1716   2406


  Latency (cycles)  DSP        BRAM        LUT            FF
------------------  ---------  ----------  -------------  --------------
            100352  150 / 220  3500 / 280  37218 / 53200  45774 / 106400



accept
*******
(2, 3)
Partition 2:
-----------

We can see the performance and resource predictions for the optimised configuration.

In [4]:
# load the optimised network
net.load_network("outputs/comms_opt.json")
net.update_partitions()

# print the performance and resource predictions
print(f"predicted latency (us): {net.get_latency()*1000000}")
print(f"predicted throughput (img/s): {net.get_throughput()} (batch size={net.batch_size})")
print(f"predicted resource usage: {net.partitions[0].get_resource_usage()}")

predicted latency (us): 1766.14
predicted throughput (img/s): 566.2065294936981 (batch size=1)
predicted resource usage: {'FF': 67241, 'LUT': 52308, 'DSP': 38, 'BRAM': 634}


We can also see a visualisation of the model. It's clear how SAMO is able to exploit parallelism to generate higher performance designs.

In [5]:
# create network visualisation
net.visualise("optimised.png", mode="png")

# display to jupyter notebook
im = Image('optimised.png')
display(im)

Once we are happy with the design, we can now generate the hardware using the [fpgaconvnet-hls](github.com/AlexMontgomerie/fpgaconvnet-hls) project. This part of the tool acts as a backend for modelling, implementing the hardware building blocks. To start with, we can instantiate the HLS code generator for this hardware configuration, and generate all the HLS hardware.

In [6]:

from fpgaconvnet.hls.generate.network import GenerateNetwork

# create instance of the network
gen_net = GenerateNetwork("lenet", "outputs/lenet_opt.json", "models/lenet-sim.onnx")

# generate hardware and create HLS project
gen_net.create_partition_project(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'michal' on host 'michal' (Linux_x86_64 version 5.4.0-125-generic) on Tue Sep 20 13:29:27 BST 2022
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/michal/Desktop/fpgaconvnet-tutorial-2022'
Sourcing Tcl script '/home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/scripts/hls/create_partition_project.tcl'
project: partition_0
INFO: [HLS 200-10] Creating and opening project '/home/michal/Desktop/fpgaconvnet-tutorial-2022/partition

For the hardware, we will need to generate a test input for the hardware, as well as a golden output.

In [7]:
import numpy as np
import PIL

# show test image
im = Image('mnist_example.png')
display(im)

# load test data
input_image = PIL.Image.open("mnist_example.png") # load file
input_image = np.array(input_image, dtype=np.float32) # convert to numpy
input_image = np.expand_dims(input_image, axis=0) # add channel dimension
input_image = input_image/np.linalg.norm(input_image) # normalise
input_image = np.stack([input_image for _ in range(1)], axis=0 ) # duplicate across batch size

Next we can test the hardware. First let's run C-Simulation, which is just a functional check of the generated hardware. To do so, we need to create a test image.

In [8]:
# run the partition's testbench
#gen_net.run_testbench(0, sample_input)

After the C-based simulation passes, the hardware can be generated. C-SYNTHESIS is the process of converting the high-level C description to RTL. We will generate the hardware for the single partition based on the `outputs/single_layer_opt.json` configuration.

It's encouraged to look through the HLS log to make sure there are no issues with pipelining or predicted timing.

In [ ]:
# generate hardware
gen_net.generate_partition_hardware(0)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'michal' on host 'michal' (Linux_x86_64 version 5.4.0-125-generic) on Tue Sep 20 13:37:16 BST 2022
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/michal/Desktop/fpgaconvnet-tutorial-2022'
Sourcing Tcl script '/home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/scripts/hls/run_csynth.tcl'
INFO: [HLS 200-10] Opening project '/home/michal/Desktop/fpgaconvnet-tutorial-2022/partition_0'.
INFO: [HLS 200-10] Opening solution '/home/

INFO: [HLS 200-10] Analyzing design file 'partition_0/src/import_conv3_Relu.cpp' ... 
In file included from partition_0/src/import_conv3_Relu.cpp:1:
In file included from partition_0/include/import_conv3_Relu.hpp:4:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/relu.hpp:21:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/common.hpp:21:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/system.hpp:4:
/tools/Xilinx/Vivado/2019.1/include/gmp.h:2226:345: warning: equality comparison with extraneous parentheses [-Wparentheses-equality]
  do { mp_size_t __gmp_i; mp_limb_t __gmp_x; __gmp_i = (__gmp_ysize); if (__gmp_i != 0) { if (__gmpn_add_n (__gmp_wp, __gmp_xp, __gmp_yp, __gmp_i)) { do { if (__gmp_i >= (__gmp_xsize)) { (__gmp_c) = 1; goto __gmp_done; } __gmp_x = (__gmp_xp)[__gmp_i]; } while 

INFO: [HLS 200-10] Analyzing design file 'partition_0/src/import_pool2_MaxPool.cpp' ... 
In file included from partition_0/src/import_pool2_MaxPool.cpp:1:
In file included from partition_0/include/import_pool2_MaxPool.hpp:4:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/sliding_window.hpp:21:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/common.hpp:21:
In file included from /home/michal/miniconda3/envs/fpgaconvnet/lib/python3.8/site-packages/fpgaconvnet/hls/hardware/system.hpp:4:
/tools/Xilinx/Vivado/2019.1/include/gmp.h:2226:345: warning: equality comparison with extraneous parentheses [-Wparentheses-equality]
  do { mp_size_t __gmp_i; mp_limb_t __gmp_x; __gmp_i = (__gmp_ysize); if (__gmp_i != 0) { if (__gmpn_add_n (__gmp_wp, __gmp_xp, __gmp_yp, __gmp_i)) { do { if (__gmp_i >= (__gmp_xsize)) { (__gmp_c) = 1; goto __gmp_done; } __gmp_x = (__gmp_xp)

Finally, we can run co-simulation. This runs RTL-level simulation of the generated circuit, and checks it against the C-simulation result.

In [10]:
# run co-simulation
#gen_net.run_cosimulation(0, sample_input)